# World Ressources Institute - CAIT data - compare data from several obtentions 

The WRI's ClimateWatch tool is already a portal towards the presentation of several datasets (CAIT, UNFCCC, GCP, PIK, etc..). It stores the data and delivers an API to provide a web user with subsets of the data he might ask for. Accordingly, a lot of piping is done by the ClimateWatch plateform when presenting the data. For the GHG emissions (CAIT and UNFCCC), the data can be retrieved from three manners:
- import all data: this provides a zipped folder with a lot of content. GHG data are provided in excel files. For CAIT, it is teh CW_CAIT_GHG_Emissions_31102017.xlsx file
- import GHG data only: this provides a zipped folder with GHG datasets only. For CAIT, it is the CW_CAIT_GHG_Emissions.xlsx file.
- use the WRI API to get the GHG data: this is the typical way to get the data, intended by the WRI community. The webuser uses the provided API to select the data (typically GHG, CAIT, and the period) he wants. For CAIT, we obtain a historical_emissions.csv file, renamed here historical_emissions_cait.csv

As WRI is already doing some data preprocessing, the datasets are not exactly the same according to the manner they have been retreived from the ClimatWatch tool.The objective of this notebook is to compare them for teh CAIT data, in order to notice the differences, and select the best one to be mapped. 

The ISO-alpha3 code is listed in countries_codes_and_coordinates.csv, taken from https://gist.github.com/tadast/8827699#file-countries_codes_and_coordinates-csv



## Import libraries and load datasets

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go

pd.set_option("max_columns", 30)
pd.set_option("max_rows", 200)

In [2]:
# CAIT emissions from raw data download
all_CAIT = pd.read_excel("../../../data/ghg-emissions/wri/CW_CAIT_GHG_Emissions_31102017.xlsx", header=1, 
                               engine="openpyxl", sheet_name="GHG Emissions")
allCO2_CAIT = pd.read_excel("../../../data/ghg-emissions/wri/CW_CAIT_GHG_Emissions_31102017.xlsx", header=0, 
                               engine="openpyxl", sheet_name="CO2 Total Emissions")

# CAIT emissions from ghg data download
ghg_CAIT = pd.read_excel("../../../data/ghg-emissions/wri/CW_CAIT_GHG_Emissions.xlsx", header=0)

# CAIT emissions from api data download
api_CAIT = pd.read_csv("../../../data/ghg-emissions/wri/historical_emissions_cait.csv", header=0)

In [3]:
all_CAIT.head()

,Country,Year,Total GHG Emissions Excluding Land-Use Change and Forestry (MtCO2e),Total GHG Emissions Including Land-Use Change and Forestry (MtCO₂e‍),Total CO2 (excluding Land-Use Change and Forestry) (MtCO2),Total CH4 (MtCO2e),Total N2O (MtCO2e),Total F-Gas (MtCO2e),Total CO2 (including Land-Use Change and Forestry) (MtCO2),Total CH4 (including Land-Use Change and Forestry) (MtCO2e),Total N2O (including Land-Use Change and Forestry) (MtCO2e),Energy (MtCO2e),Industrial Processes (MtCO2e),Agriculture (MtCO2e),Waste (MtCO2e),Land-Use Change and Forestry (MtCO2),Bunker Fuels (MtCO2),Electricity/Heat (MtCO2),Manufacturing/Construction (MtCO2),Transportation (MtCO2),Other Fuel Combustion (MtCO2e),Fugitive Emissions (MtCO2e)
0,Afghanistan,1990,15.212848,15.212848,2.915024,9.311589,2.984055,0.002180,2.915024,9.311589,2.984055,3.774044,0.057140,7.342710,4.038954,0.0,NaN,NaN,NaN,NaN,NaN,1.242220
1,Afghanistan,1991,15.286439,15.286439,2.684445,9.516187,3.082194,0.003613,2.684445,9.516187,3.082194,3.376803,0.058573,7.631027,4.220036,0.0,NaN,NaN,NaN,NaN,NaN,1.020851
2,Afghanistan,1992,14.010531,14.010531,1.392269,9.571483,3.041733,0.005046,1.392269,9.571483,3.041733,1.914300,0.063670,7.631443,4.401118,0.0,NaN,NaN,NaN,NaN,NaN,0.602588
3,Afghanistan,1993,14.028118,14.028118,1.322704,9.609869,3.089066,0.006479,1.322704,9.609869,3.089066,1.678073,0.065103,7.702742,4.582200,0.0,NaN,NaN,NaN,NaN,NaN,0.413993
4,Afghanistan,1994,13.985408,13.985408,1.267744,9.789039,2.920713,0.007912,1.267744,9.789039,2.920713,1.456451,0.066536,7.699139,4.763283,0.0,NaN,NaN,NaN,NaN,NaN,0.247331


In [4]:
ghg_CAIT.head()

,Country,Source,Sector,Gas,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,CAIT,Total excluding LUCF,All GHG,15.182850,15.102010,13.634690,13.463990,13.271730,13.476050,14.437570,15.342910,16.095230,16.914580,15.075750,...,16.515810,17.398280,21.038220,24.878710,31.537650,36.955460,44.906160,58.651860,66.749280,74.796110,84.61923,93.728620,95.372840,97.300110,98.92076
1,AFG,CAIT,Total including LUCF,All GHG,12.794040,12.713210,11.245880,11.075190,10.882930,11.087250,12.048770,12.954110,13.706420,14.525780,12.686940,...,16.637720,17.520180,21.160120,25.000610,31.659550,37.077360,45.028070,58.405640,66.503060,74.549890,84.37301,93.482400,95.527500,97.454770,99.07541
2,AFG,CAIT,Energy,All GHG,5.829497,5.334624,3.760858,3.422760,3.102594,2.783429,2.651769,2.509109,2.389449,2.096789,2.017130,...,2.271723,2.759622,6.319421,10.001220,15.278020,20.140820,26.056620,39.529420,47.592230,55.527030,64.67584,74.741640,75.932910,77.718180,79.58044
3,AFG,CAIT,Industrial Processes,All GHG,0.051879,0.054500,0.060111,0.062722,0.065343,0.067964,0.081694,0.095434,0.109174,0.122915,0.109985,...,0.142366,0.142436,0.162460,0.176814,0.202657,0.222971,0.248895,0.313896,0.378967,0.449909,0.53463,0.592081,0.758807,0.911544,1.06428
4,AFG,CAIT,Agriculture,All GHG,8.072853,8.396465,8.409491,8.486480,8.523959,8.957016,9.977472,10.952730,11.751970,12.791240,10.986000,...,11.769860,12.072050,12.054550,12.121270,13.399950,13.857020,15.788380,15.902260,15.777790,15.724860,16.22045,15.112570,15.315740,15.221950,14.74454


In [5]:
api_CAIT.head()

,Country,Data source,Sector,Gas,Unit,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,World,CAIT,Total including LUCF,All GHG,MtCO₂e,48939.71,47990.47,47413.95,46760.47,46647.29,46047.13,45427.61,44891.40,44758.58,43029.01,...,39437.14,37618.01,36727.84,35701.77,35607.73,34948.94,34929.19,35387.89,34068.91,33703.46,32977.47,32766.49,32670.32,32813.46,32645.91
1,China,CAIT,Total including LUCF,All GHG,MtCO₂e,11705.81,11408.26,11207.66,11149.68,11155.76,11144.76,10690.04,10364.83,9872.37,9046.19,...,6135.95,5386.89,4769.03,4459.91,4249.70,4053.37,4103.23,3963.61,3954.75,3918.30,3528.81,3377.79,3154.54,3023.38,2873.71
2,United States,CAIT,Total including LUCF,All GHG,MtCO₂e,5794.35,5613.64,5676.92,5586.69,5711.15,5690.79,5550.87,5796.78,6041.59,5793.66,...,6387.22,6306.12,6246.95,6406.04,6446.20,6291.25,6293.80,6258.47,6010.62,5843.90,5782.20,5691.26,5583.80,5501.92,5543.47
3,India,CAIT,Total including LUCF,All GHG,MtCO₂e,3346.63,3202.82,3073.24,3002.17,2988.34,2816.49,2758.53,2610.32,2576.93,2467.78,...,1905.13,1817.42,1769.79,1747.49,1498.20,1460.24,1382.26,1348.09,1289.63,1240.44,1174.78,1128.44,1096.71,1064.49,1009.44
4,European Union (27),CAIT,Total including LUCF,All GHG,MtCO₂e,3333.16,3401.95,3387.97,3046.38,2990.49,3136.34,3213.21,3274.40,3646.50,3566.88,...,4005.19,4008.81,3921.02,3940.21,3933.88,3934.85,4008.29,4048.58,4126.40,4018.79,3968.32,3984.95,4058.11,4202.13,4279.18


Data is not stored according to the same parameters + country are not referenced accordingly + the years extend to 2018 when obtained from api and ghg + precision is smaller for API + only total including LUCF has been obtained from the API. 

In [6]:
print(sorted(set(api_CAIT["Country"].unique()) - set(all_CAIT["Country"].unique()) ))
print()
print(set(all_CAIT["Country"].unique()) - set(api_CAIT["Country"].unique()) )

['Antigua and Barbuda', 'Bosnia and Herzegovina', "Côte d'Ivoire", 'Democratic Republic of the Congo', 'Eswatini', 'European Union (27)', 'Macedonia', 'North Korea', 'Republic of Congo', 'Russia', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines', 'Sao Tome and Principe', 'South Korea', 'South Sudan', 'Timor-Leste', 'Trinidad and Tobago', 'United States']

{'European Union (28)', 'Trinidad & Tobago', 'Korea (North)', 'Saint Kitts & Nevis', 'Bosnia & Herzegovina', 'Macedonia, FYR', 'Congo, Dem. Republic', 'Congo', "Cote d'Ivoire", 'Sao Tome & Principe', 'Swaziland', 'United States of America', 'Saint Vincent & Grenadines', 'Russian Federation', 'Korea (South)', 'Antigua & Barbuda'}


In [7]:
#dictionnary of variables 
dict_corr = {
    'Antigua and Barbuda': 'Antigua & Barbuda', 
    'Bosnia and Herzegovina': 'Bosnia & Herzegovina',
    "Côte d'Ivoire": "Cote d'Ivoire", 
    'Democratic Republic of the Congo': 'Congo, Dem. Republic', 
    'Eswatini': 'Swaziland', 
    'Macedonia': 'Macedonia, FYR', 
    'North Korea': 'Korea (North)', 
    'Republic of Congo': 'Congo',
    'Russia': 'Russian Federation',
    'Saint Kitts and Nevis': 'Saint Kitts & Nevis', 
    'Saint Vincent and the Grenadines': 'Saint Vincent & Grenadines', 
    'Sao Tome and Principe':  'Sao Tome & Principe', 
    'South Korea': 'Korea (South)', 
    'Trinidad and Tobago': 'Trinidad & Tobago', 
    'United States': 'United States of America' 
}

#missing in all, present in api: South Sudan, EU(28), Timor Leste
#missing in api, present in all: EU27 ()

In [8]:
ghg_CAIT["Country"].unique()

array(['AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS',
       'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR',
       'BHS', 'BIH', 'BLR', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN',
       'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD',
       'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE',
       'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI',
       'ESP', 'EST', 'ETH', 'EUU', 'FIN', 'FJI', 'FRA', 'FSM', 'GAB',
       'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD',
       'GTM', 'GUY', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IND', 'IRL',
       'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ',
       'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN',
       'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA',
       'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLI', 'MLT',
       'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM',
       'NER', 'NGA',

In [9]:
#Let's just convert everything into the ALPHA ISO-3 data 

#import correspondance tabular
corr_iso_country = pd.read_csv("countries_codes_and_coordinates.csv", sep=';', skipinitialspace=True)
corr_iso_country

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6
...,...,...,...,...,...,...
251,Wallis and Futuna,WF,WLF,876,-13.3000,-176.2
252,Western Sahara,EH,ESH,732,24.5000,-13.0
253,Yemen,YE,YEM,887,15.0000,48.0
254,Zambia,ZM,ZMB,894,-15.0000,30.0


In [10]:
#difference between ghg_CAIT and this table 
print(set(ghg_CAIT["Country"].unique()) - set(corr_iso_country["Alpha-3 code"].unique()))
print(set(corr_iso_country["Alpha-3 code"].unique()) - set(ghg_CAIT["Country"].unique()) )

#ok, it's almost enough


{'EUU', 'WORLD'}
{'REU', 'WLF', 'TKL', 'AIA', 'CYM', 'SGS', 'SMR', 'JEY', 'SHN', 'VIR', 'MNP', 'BVT', 'MCO', 'PYF', 'MTQ', 'ESH', 'MYT', 'CXR', 'GLP', 'GIB', 'MSR', 'ATA', 'CCK', 'PRI', 'ASM', 'NFK', 'TCA', 'ATF', 'ANT', 'PCN', 'BMU', 'HMD', 'IMN', 'PSE', 'MAC', 'HKG', 'GRL', 'TWN', 'SPM', 'UMI', 'VAT', 'FLK', 'IOT', 'GGY', 'SJM', 'GUF', 'GUM', 'ABW', 'FRO', 'VGB', 'NCL'}


In [11]:
#difference between api_CAIT and this table 
print(sorted(set(api_CAIT["Country"].unique()) - set(corr_iso_country["Country"].unique())))
print()
print(sorted(set(corr_iso_country["Country"].unique()) - set(api_CAIT["Country"].unique()) ))

#almost ok

['Democratic Republic of the Congo', 'Eswatini', 'European Union (27)', 'Iran', 'Laos', 'Macedonia', 'Micronesia', 'Moldova', 'North Korea', 'Republic of Congo', 'Syria', 'Tanzania', 'World']

['American Samoa', 'Anguilla', 'Antarctica', 'Aruba', 'Bermuda', 'Bolivia, Plurinational State of', 'Bouvet Island', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Burma', 'Cayman Islands', 'Christmas Island', 'Cocos (Keeling) Islands', 'Congo', 'Congo, the Democratic Republic of the', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Hong Kong', 'Iran, Islamic Republic of', 'Isle of Man', 'Ivory Coast', 'Jersey', "Korea, Democratic People's Republic of", 'Korea, Republic of', "Lao People's Democratic Republic", 'Libyan Arab Jamahiriya', 'Macao', 'Macedonia, the former Yugoslav Republic of', 'M

In [12]:
#difference between all_CAIT and this table 
print(sorted(set(all_CAIT["Country"].unique()) - set(corr_iso_country["Country"].unique())))
print()
print(sorted(set(corr_iso_country["Country"].unique()) - set(all_CAIT["Country"].unique()) ))

#almost ok

['Antigua & Barbuda', 'Bosnia & Herzegovina', 'Congo, Dem. Republic', "Cote d'Ivoire", 'European Union (28)', 'Iran', 'Korea (North)', 'Korea (South)', 'Laos', 'Macedonia, FYR', 'Micronesia', 'Moldova', 'Saint Kitts & Nevis', 'Saint Vincent & Grenadines', 'Sao Tome & Principe', 'Syria', 'Tanzania', 'Trinidad & Tobago', 'United States of America', 'World']

['American Samoa', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Aruba', 'Bermuda', 'Bolivia, Plurinational State of', 'Bosnia and Herzegovina', 'Bouvet Island', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Burma', 'Cayman Islands', 'Christmas Island', 'Cocos (Keeling) Islands', 'Congo, the Democratic Republic of the', "Côte d'Ivoire", 'Falkland Islands (Malvinas)', 'Faroe Islands', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Hong Kong', 'Iran, Islamic Republic of', 

# CONCLUSION 

It is possible to turn all the countries into the ALPHA ISO column. We add a line with corresponding ISO in the referential

In [13]:
#Defined in all CAIT but missing in ISO
to_add = {'Antigua & Barbuda' : 'ATG',
'Bosnia & Herzegovina': 'BIH',
'Congo, Dem. Republic': 'COD',
"Cote d'Ivoire": 'CIV',
'European Union (28)': 'EUU',
'Iran': 'IRN',
'Korea (North)': 'PRK',
'Korea (South)': 'KOR',
'Laos': 'LAO',
'Macedonia, FYR': 'MKD',
'Micronesia': 'FSM',
'Moldova': 'MDA',
'Saint Kitts & Nevis': 'KNA',
'Saint Vincent & Grenadines': 'VCT',
'Sao Tome & Principe': 'STP',
'Syria' : 'SYR',
'Tanzania': 'TZA',
'Trinidad & Tobago': 'TTO',
'United States of America': 'USA',
'World' : 'WORLD',
#Defined in api CAIT but missing in ISO
'Democratic Republic of the Congo': 'COD',
'Eswatini': 'SWZ',
'European Union (27)': 'EUU',
#'Iran',
#'Laos', 
'Macedonia': 'MKD',
#'Micronesia', 
#'Moldova', 
'North Korea': 'PRK',
'Republic of Congo': 'COG'
#'Syria',
#'Tanzania',
#'World'
}

#add line in corr_completed_iso
completed_iso = corr_iso_country.copy()
rows_list = []
for x in to_add:
    rows_list.append([x, np.nan, to_add[x], np.nan, np.nan, np.nan])                
completed_iso = completed_iso.append(pd.DataFrame(rows_list, columns=corr_iso_country.columns), ignore_index=True)

In [14]:
completed_iso

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6
...,...,...,...,...,...,...
277,Eswatini,NaN,SWZ,NaN,NaN,NaN
278,European Union (27),NaN,EUU,NaN,NaN,NaN
279,Macedonia,NaN,MKD,NaN,NaN,NaN
280,North Korea,NaN,PRK,NaN,NaN,NaN


All countries in all_cait and api_cait can be turned into an ISO identifier. We add a column in the corresponding datasets

In [15]:
all_CAIT["alpha_3"] = [completed_iso[completed_iso["Country"] == country].iloc[0,2] for country in all_CAIT["Country"]]
api_CAIT["alpha_3"] = [completed_iso[completed_iso["Country"] == country].iloc[0,2] for country in api_CAIT["Country"]]

,Country,Source,Sector,Gas,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,CAIT,Total excluding LUCF,All GHG,15.182850,15.102010,13.634690,13.463990,13.271730,13.476050,14.437570,15.342910,16.095230,16.914580,15.075750,...,16.515810,17.398280,21.038220,24.878710,31.537650,36.955460,44.906160,58.651860,66.749280,74.796110,84.61923,93.728620,95.372840,97.300110,98.920760
1,AFG,CAIT,Total including LUCF,All GHG,12.794040,12.713210,11.245880,11.075190,10.882930,11.087250,12.048770,12.954110,13.706420,14.525780,12.686940,...,16.637720,17.520180,21.160120,25.000610,31.659550,37.077360,45.028070,58.405640,66.503060,74.549890,84.37301,93.482400,95.527500,97.454770,99.075410
2,AFG,CAIT,Energy,All GHG,5.829497,5.334624,3.760858,3.422760,3.102594,2.783429,2.651769,2.509109,2.389449,2.096789,2.017130,...,2.271723,2.759622,6.319421,10.001220,15.278020,20.140820,26.056620,39.529420,47.592230,55.527030,64.67584,74.741640,75.932910,77.718180,79.580440
3,AFG,CAIT,Industrial Processes,All GHG,0.051879,0.054500,0.060111,0.062722,0.065343,0.067964,0.081694,0.095434,0.109174,0.122915,0.109985,...,0.142366,0.142436,0.162460,0.176814,0.202657,0.222971,0.248895,0.313896,0.378967,0.449909,0.53463,0.592081,0.758807,0.911544,1.064280
4,AFG,CAIT,Agriculture,All GHG,8.072853,8.396465,8.409491,8.486480,8.523959,8.957016,9.977472,10.952730,11.751970,12.791240,10.986000,...,11.769860,12.072050,12.054550,12.121270,13.399950,13.857020,15.788380,15.902260,15.777790,15.724860,16.22045,15.112570,15.315740,15.221950,14.744540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9077,ZWE,CAIT,Other Fuel Combustion,N2O,0.407812,0.417100,0.426387,0.435675,0.444962,0.587562,0.730026,0.872489,1.014953,1.157417,1.299881,...,0.883322,0.779182,0.803597,0.828012,0.852426,0.876841,0.901256,1.023247,1.145238,1.267229,1.38922,1.511210,1.536870,1.562529,1.588188
9078,ZWE,CAIT,Fugitive Emissions,N2O,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
9079,ZWE,CAIT,Total excluding LUCF,F-Gas,0.067604,0.064739,0.061875,0.059011,0.056146,0.053282,0.072341,0.091400,0.110460,0.129519,0.148578,...,0.195809,0.207617,0.230892,0.254168,0.277443,0.300719,0.323994,0.379388,0.434782,0.490176,0.54557,0.600965,0.665562,0.730160,0.794758
9080,ZWE,CAIT,Total including LUCF,F-Gas,0.067604,0.064739,0.061875,0.059011,0.056146,0.053282,0.072341,0.091400,0.110460,0.129519,0.148578,...,0.195809,0.207617,0.230892,0.254168,0.277443,0.300719,0.323994,0.379388,0.434782,0.490176,0.54557,0.600965,0.665562,0.730160,0.794758


In [17]:
# Comapre GHG CAIT and API CAIT
ghg_CAIT_filtered = ghg_CAIT[(ghg_CAIT["Sector"] == "Total including LUCF") & (ghg_CAIT["Gas"] == "All GHG")]
ghg_CAIT_filtered.rename(columns={'Country' : 'alpha_3', 'Source': 'Data Source'}, inplace=True)
ghg_CAIT_filtered.reset_index(inplace=True, drop=True)
ghg_CAIT_filtered = ghg_CAIT_filtered.sort_values(by="alpha_3").reset_index(drop=True)
ghg_CAIT_filtered

,alpha_3,Data Source,Sector,Gas,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,CAIT,Total including LUCF,All GHG,12.794040,12.713210,11.245880,11.075190,10.882930,11.087250,12.048770,12.954110,13.706420,14.525780,12.686940,...,16.637720,17.520180,21.160120,25.000610,31.659550,37.077360,45.028070,58.405640,66.503060,74.549890,84.373010,93.482400,95.527500,97.454770,99.075410
1,AGO,CAIT,Total including LUCF,All GHG,65.179160,65.845540,66.170630,66.958000,67.102960,71.466140,73.329470,71.740410,79.624930,77.645270,73.287940,...,114.918700,115.137200,114.992800,121.856700,123.385000,128.341800,133.354700,133.542900,133.206500,135.410300,136.497900,139.085700,138.854500,137.317200,124.591900
2,ALB,CAIT,Total including LUCF,All GHG,11.509050,9.147693,7.091174,7.032104,7.871299,7.630235,7.355607,6.645458,6.878919,8.092082,8.484637,...,8.192747,7.978511,7.878905,7.944576,7.668920,7.732635,8.012203,8.792089,8.399308,8.596466,9.082266,9.008917,9.485248,10.059020,9.841821
3,AND,CAIT,Total including LUCF,All GHG,0.430703,0.432506,0.434309,0.439776,0.437915,0.458038,0.493621,0.510884,0.542803,0.571058,0.588321,...,0.643489,0.662777,0.638078,0.635363,0.639976,0.622605,0.627218,0.602272,0.599310,0.589020,0.575066,0.579432,0.586052,0.585343,0.587181
4,ARE,CAIT,Total including LUCF,All GHG,69.710520,76.253090,74.823840,77.757550,84.366470,90.780960,96.088560,103.860100,108.391300,112.690300,113.343800,...,148.024800,156.704400,162.952500,174.274400,199.890500,202.076400,208.579300,214.634200,226.567100,237.631100,240.020600,252.852500,259.645700,269.822000,263.239400
5,ARG,CAIT,Total including LUCF,All GHG,288.373700,293.361700,297.755600,300.268900,305.349500,305.933600,319.809200,322.335900,326.148100,332.245000,331.144900,...,406.443900,410.920400,425.416400,438.824100,442.975600,424.071000,429.808600,410.639700,414.838400,424.702500,421.758300,428.374900,396.269500,397.697800,395.498600
6,ARM,CAIT,Total including LUCF,All GHG,24.442540,25.067660,14.952270,8.765833,6.238958,6.846310,5.778699,6.388588,6.312973,5.801620,6.153152,...,6.748061,7.468687,7.899751,8.564243,9.114554,8.020746,7.875077,8.300317,9.286578,9.222494,9.184014,9.177997,9.541392,9.318840,9.397270
7,ATG,CAIT,Total including LUCF,All GHG,0.359410,0.372226,0.450396,0.427165,0.433083,0.459654,0.485055,0.520347,0.568340,0.588586,0.609162,...,0.757573,0.788464,0.859508,0.917346,0.970969,1.926584,1.039144,1.066380,1.286581,1.056556,1.070591,1.095618,1.134099,1.160076,1.212433
8,AUS,CAIT,Total including LUCF,All GHG,558.209500,558.133700,560.296500,559.983600,563.056400,571.190500,571.006800,582.279500,608.082500,643.600800,661.970600,...,635.502600,598.439200,646.905700,646.462500,617.089000,633.804300,601.139900,644.735100,643.116600,555.261200,566.211500,566.469300,577.317800,623.079900,619.263900
9,AUT,CAIT,Total including LUCF,All GHG,62.788170,66.993820,61.202930,61.122330,61.215790,63.693630,67.412460,66.398260,66.380680,64.557200,64.764890,...,83.588930,83.629860,81.307030,78.224950,77.788490,70.834500,76.554700,73.724660,70.504410,71.053500,67.280150,68.407800,68.534220,70.709180,67.854630


In [18]:
api_CAIT_filtered = api_CAIT.copy()
api_CAIT_filtered.drop(["Country", "Unit"], axis=1, inplace=True)
api_CAIT_filtered.columns = ["Data Source", "Sector", "Gas"] + [2018 - i for i in range(29)] + ["alpha_3"]
api_CAIT_filtered = api_CAIT_filtered[ghg_CAIT_filtered.columns]
api_CAIT_filtered = api_CAIT_filtered.sort_values(by="alpha_3").reset_index(drop=True)
api_CAIT_filtered

,alpha_3,Data Source,Sector,Gas,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,CAIT,Total including LUCF,All GHG,12.79,12.71,11.25,11.08,10.88,11.09,12.05,12.95,13.71,14.53,12.69,...,16.64,17.52,21.16,25.00,31.66,37.08,45.03,58.41,66.50,74.55,84.37,93.48,95.53,97.45,99.08
1,AGO,CAIT,Total including LUCF,All GHG,65.18,65.85,66.17,66.96,67.10,71.47,73.33,71.74,79.62,77.65,73.29,...,114.92,115.14,114.99,121.86,123.39,128.34,133.35,133.54,133.21,135.41,136.50,139.09,138.85,137.32,124.59
2,ALB,CAIT,Total including LUCF,All GHG,11.51,9.15,7.09,7.03,7.87,7.63,7.36,6.65,6.88,8.09,8.48,...,8.19,7.98,7.88,7.94,7.67,7.73,8.01,8.79,8.40,8.60,9.08,9.01,9.49,10.06,9.84
3,AND,CAIT,Total including LUCF,All GHG,0.43,0.43,0.43,0.44,0.44,0.46,0.49,0.51,0.54,0.57,0.59,...,0.64,0.66,0.64,0.64,0.64,0.62,0.63,0.60,0.60,0.59,0.58,0.58,0.59,0.59,0.59
4,ARE,CAIT,Total including LUCF,All GHG,69.71,76.25,74.82,77.76,84.37,90.78,96.09,103.86,108.39,112.69,113.34,...,148.02,156.70,162.95,174.27,199.89,202.08,208.58,214.63,226.57,237.63,240.02,252.85,259.65,269.82,263.24
5,ARG,CAIT,Total including LUCF,All GHG,288.37,293.36,297.76,300.27,305.35,305.93,319.81,322.34,326.15,332.24,331.14,...,406.44,410.92,425.42,438.82,442.98,424.07,429.81,410.64,414.84,424.70,421.76,428.37,396.27,397.70,395.50
6,ARM,CAIT,Total including LUCF,All GHG,24.44,25.07,14.95,8.77,6.24,6.85,5.78,6.39,6.31,5.80,6.15,...,6.75,7.47,7.90,8.56,9.11,8.02,7.88,8.30,9.29,9.22,9.18,9.18,9.54,9.32,9.40
7,ATG,CAIT,Total including LUCF,All GHG,0.36,0.37,0.45,0.43,0.43,0.46,0.49,0.52,0.57,0.59,0.61,...,0.76,0.79,0.86,0.92,0.97,1.93,1.04,1.07,1.29,1.06,1.07,1.10,1.13,1.16,1.21
8,AUS,CAIT,Total including LUCF,All GHG,558.21,558.13,560.30,559.98,563.06,571.19,571.01,582.28,608.08,643.60,661.97,...,635.50,598.44,646.91,646.46,617.09,633.80,601.14,644.74,643.12,555.26,566.21,566.47,577.32,623.08,619.26
9,AUT,CAIT,Total including LUCF,All GHG,62.79,66.99,61.20,61.12,61.22,63.69,67.41,66.40,66.38,64.56,64.76,...,83.59,83.63,81.31,78.22,77.79,70.83,76.55,73.72,70.50,71.05,67.28,68.41,68.53,70.71,67.85


In [19]:
api_CAIT_filtered.compare(ghg_CAIT_filtered)

1990                  1991                  1992               \
        self        other     self        other     self        other   
0      12.79    12.794040    12.71    12.713210    11.25    11.245880   
1      65.18    65.179160    65.85    65.845540    66.17    66.170630   
2      11.51    11.509050     9.15     9.147693     7.09     7.091174   
3       0.43     0.430703     0.43     0.432506     0.43     0.434309   
4      69.71    69.710520    76.25    76.253090    74.82    74.823840   
5     288.37   288.373700   293.36   293.361700   297.76   297.755600   
6      24.44    24.442540    25.07    25.067660    14.95    14.952270   
7       0.36     0.359410     0.37     0.372226     0.45     0.450396   
8     558.21   558.209500   558.13   558.133700   560.30   560.296500   
9      62.79    62.788170    66.99    66.993820    61.20    61.202930   
10     75.17    75.171600    84.19    84.193940    60.44    60.442140   
11      8.33     8.334656     8.44     8.438004     8.48     8.475379   
12    126.81   126.813300   131.76   131.756200   129.84   129.841600   
13     20.31    20.311850    20.44    20.441080    20.56    20.560340   
14     26.04    26.042430    26.49    26.485770    26.79    26.792510   
15    115.06   115.059300   115.24   115.240600   117.14   117.141200   
16     76.59    76.590850    59.37    59.368600    53.50    53.503090   
17     14.06    14.056870    13.76    13.757020    15.78    15.778610   
18      2.04     2.035854     2.14     2.137488     2.80     2.796560   
19     33.41    33.408060    30.87    30.873020    23.34    23.339400   
20    104.36   104.357200    99.30    99.300550    85.47    85.473890   
21     13.20    13.198270    13.41    13.411780    13.63    13.629340   
22    104.98   104.983800   105.78   105.775300   107.60   107.600400   
23   1642.29  1642.292000  1669.56  1669.565000  1679.50  1679.498000   
24      3.12     3.119853     3.15     3.154315     3.39     3.388226   
25     15.51    15.506090    15.11    15.113020    15.45    15.453910   
26     -5.46    -5.459410    -5.41    -5.408740    -5.42    -5.420190   
27     54.94    54.943600    54.38    54.382270    54.65    54.645420   
28     64.44    64.437670    65.19    65.191520    65.67    65.673270   
29    626.36   626.362100   622.29   622.294500   638.22   638.223300   
30     51.19    51.190320    53.40    53.395320    53.31    53.309440   
31     30.34    30.341710    29.62    29.616100    31.95    31.954380   
32   2873.71  2873.707000  3023.38  3023.381000  3154.54  3154.538000   
33     69.67    69.672720    75.40    75.398460    80.99    80.990750   
34     71.93    71.925500    71.62    71.618900    71.57    71.569560   
35    434.87   434.869300   433.76   433.761900   434.16   434.156000   
36     13.88    13.881220    16.10    16.100380    16.01    16.014560   
37      0.03     0.025736     0.03     0.025738     0.03     0.026277   
38    217.08   217.084400   218.85   218.850400   222.51   222.512000   
39      0.34     0.335101     0.34     0.336337     0.34     0.344936   
40     -0.27    -0.272470    -0.26    -0.259330    -0.22    -0.223820   
41     11.34    11.340690    11.61    11.606210    12.71    12.710030   
42     43.70    43.701370    34.84    34.835870    28.90    28.900740   
43      5.35     5.345394     5.87     5.868455     6.29     6.286570   
44    172.90   172.901900   156.17   156.174600   151.72   151.724700   
45       NaN          NaN  1079.69  1079.694000  1032.01  1032.006000   
46      0.99     0.988822     1.01     1.012147     1.02     1.018189   
47      0.24     0.243577     0.24     0.244681     0.26     0.263480   
48     68.51    68.508160    79.13    79.131710    72.94    72.935630   
49      8.65     8.645446     9.80     9.801090    11.31    11.309850   
50     95.67    95.666920    94.54    94.536730    94.16    94.161610   
51     70.40    70.399260    71.25    71.248060    73.51    73.508690   
52    129.43   129.428800   135.07   135.074300   140.18   140.176

In [20]:
#It seems correct. Using the API, precise data has been lost in the process. 